## Import Thư Viện Xử Lý

In [ ]:
import requests # Lấy đoạn mã html của trang web
import time # Thời gian chờ cho mỗi request
import pandas as pd # Dữ Liệu

from bs4 import BeautifulSoup # Để lấy những tag trong html
import re # Xử lý những đoạn văn bản đặc biết

from tqdm import tqdm # Hiển thị thời gian xử lý
import gc # Garbage Collector, xoá đi bộ nhớ tạm (ram)

import json

In [ ]:
def create_data_url(): # Khởi tạo dữ liệu chuẩn để lấy đường link book
    book_title = []
    book_url = []
    author_name = []
    Author_Url = []
    
    return book_title, book_url, author_name, Author_Url

## Để lấy đường link của quyển sách, bài crawl được chia làm 3 hướng:
#### Lựa chọn theo thể loại (genre)
#### Lựa chọn theo Thời gian (Tháng, năm)
#### Lựa chọn theo Album
#### Lựa chọn theo Tác giả (dẫn đến đường link của mỗi tác giả để lấy danh sách books)

## Lựa chọn theo thể loại

In [ ]:
def get_Url_Book_by_Genre(genre): # Lấy theo genre, có tổng cộng 27 genre khác nhau
    source = requests.get("https://www.goodreads.com/shelf/show/{}".format(genre))
    # Ví dụ về đường dẫn: https://www.goodreads.com/shelf/show/art
    soup =   BeautifulSoup(source.text, 'html.parser')
    temp = soup.find("div", {"class": "leftContainer"}) # Lấy phần thân của đoạn cần crawl
    temp = temp.findAll("div", {"class": "elementList"}) # Lấy từng thành phần quyển sách
    
    Book_Titles, Book_Urls, Author_Names, Author_Urls = create_data_url()
    
    for t in temp:
        Book_Titles.append(str(t.find("a", {"class": "bookTitle"}).contents[0])) # Tiêu đề sách
        Book_Urls.append(str("https://www.goodreads.com" + t.find("a", {"class": "bookTitle"}).get("href"))) # Đường Link
        Author_Names.append(str(t.find("a", {"class": "authorName"}).find("span").contents[0])) # Tên tác giả
        Author_Urls.append(str(t.find("a", {"class": "authorName"}).get("href"))) # Đường link đến tác giả
    
    return pd.DataFrame({"Book_Titles": Book_Titles, "Book_Urls": Book_Urls, "Author_Names": Author_Names, "Author_Urls": Author_Urls})

In [ ]:
genres = ["art", "biography", "business", "childrens", "christian", "classics", "comics", "cookbooks", "ebooks",
         "fantasy", "fiction", "graphic-novels", "historical-fiction", "history", "horror", "memoir", "music", "mystery",
         "non-fiction", "poetry", "psychology", "romance", "science", "science-fiction", "self-help", "sports",
         "thriller", "travel", "young-adult"]

In [ ]:
# Khởi tạo dataframe ban đầu để lấy
Info_Book_Url = pd.DataFrame({"Book_Titles": [], "Book_Urls": [], "Author_Names": [], "Author_Urls": []})

for genre in tqdm(genres):
    Info_Book_Url = pd.concat([Info_Book_Url, get_Url_Book_by_Genre(genre)])
    
Info_Book_Url

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:19<00:00,  2.76s/it]


,Book_Titles,Book_Urls,Author_Names,Author_Urls,Genre
0,Ways of Seeing (Paperback),https://www.goodreads.com/book/show/2784.Ways_...,John Berger,https://www.goodreads.com/author/show/29919.Jo...,art
1,The Story of Art (Hardcover),https://www.goodreads.com/book/show/222078.The...,E.H. Gombrich,https://www.goodreads.com/author/show/167112.E...,art
2,The New Drawing on the Right Side of the Brain...,https://www.goodreads.com/book/show/627206.The...,Betty Edwards,https://www.goodreads.com/author/show/42010.Be...,art
3,Steal Like an Artist: 10 Things Nobody Told Yo...,https://www.goodreads.com/book/show/13099738-s...,Austin Kleon,https://www.goodreads.com/author/show/2985039....,art
4,Wall and Piece (Paperback),https://www.goodreads.com/book/show/114683.Wal...,Banksy,https://www.goodreads.com/author/show/28811.Ba...,art
...,...,...,...,...,...
45,"Shadow and Bone (The Shadow and Bone Trilogy, #1)",https://www.goodreads.com/book/show/10194157-s...,Leigh Bardugo,https://www.goodreads.com/author/show/4575289....,young-adult
46,"Six of Crows (Six of Crows, #1)",https://www.goodreads.com/book/show/23437156-s...,Leigh Bardugo,https://www.goodreads.com/author/show/4575289....,young-adult
47,"Scarlet (The Lunar Chronicles, #2)",https://www.goodreads.com/book/show/13206760-s...,Marissa Meyer,https://www.goodreads.com/author/show/4684322....,young-adult
48,"Graceling (Graceling Realm, #1)",https://www.goodreads.com/book/show/3236307-gr...,Kristin Cashore,https://www.goodreads.com/author/show/1373880....,young-adult


## Lựa chọn theo thời gian

In [ ]:
def get_Url_Book_by_Date(month, year): # Lấy Url Book theo thời gian (tháng, năm)
    source = requests.get("https://www.goodreads.com/book/popular_by_date/{}/{}".format(year, month))
    # Ví dụ về đường dẫn: https://www.goodreads.com/book/popular_by_date/2020/January
    soup =   BeautifulSoup(source.text, 'html.parser')

    temp = soup.find("table", {"class": "tableList"})
    temp = temp.findAll("tr", {"itemtype": "http://schema.org/Book"}) # Lấy danh sách từng quyển sách
    
    Book_Titles, Book_Urls, Author_Names, Author_Urls = create_data_url()
    
    for t in temp:
        Book_Titles.append(str(t.find("a", {"class": "bookTitle"}).find("span").contents[0]))
        Book_Urls.append(str("https://www.goodreads.com" + t.find("a", {"class": "bookTitle"}).get("href")))
        Author_Names.append(str(t.find("a", {"class": "authorName"}).find("span").contents[0]))
        Author_Urls.append(str(t.find("a", {"class": "authorName"}).get("href")))

    return pd.DataFrame({"Book_Titles": Book_Titles, "Book_Urls": Book_Urls, "Author_Names": Author_Names, "Author_Urls": Author_Urls})

In [ ]:
Months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
for month in tqdm (Months): # Jan-2019 -> Dec-2021
    for year in [2019,2020,2021]:
        Info_Book_Url = pd.concat([Info_Book_Url, get_Url_Book_by_Date(month, year)])

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [04:07<00:00, 20.65s/it]


In [ ]:
for year in tqdm(range(1920, 2021)): # Lấy từ năm 1920 -> 2020
    Info_Book_Url = pd.concat([Info_Book_Url, get_Url_Book_by_Date("", year)])

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [12:23<00:00,  7.36s/it]


## Lựa chọn theo Album

In [ ]:
def get_Url_Book_by_Album(url, page): # Lấy Url Book theo Album
    source = requests.get("{}?page={}".format(url, page))
    # Ví dụ về đường dẫn: https://www.goodreads.com/list/show/15.Best_Historical_Fiction?page=2
    soup =   BeautifulSoup(source.text, 'html.parser')

    temp = soup.find("table", {"class": "tableList"})
    temp = temp.findAll("tr", {"itemtype": "http://schema.org/Book"})
    
    Book_Titles, Book_Urls, Author_Names, Author_Urls = create_data_url()
    
    for t in temp:
        Book_Titles.append(str(t.find("a", {"class": "bookTitle"}).find("span").contents[0]))
        Book_Urls.append(str("https://www.goodreads.com" + t.find("a", {"class": "bookTitle"}).get("href")))
        Author_Names.append(str(t.find("a", {"class": "authorName"}).find("span").contents[0]))
        Author_Urls.append(str(t.find("a", {"class": "authorName"}).get("href")))

        
    return pd.DataFrame({"Book_Titles": Book_Titles, "Book_Urls": Book_Urls, "Author_Names": Author_Names, "Author_Urls": Author_Urls})

In [ ]:
list_album = ["https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once",
             "https://www.goodreads.com/list/show/1043.Books_That_Should_Be_Made_Into_Movies",
             "https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s",
             "https://www.goodreads.com/list/show/15.Best_Historical_Fiction",
             "https://www.goodreads.com/list/show/47.Best_Dystopian_and_Post_Apocalyptic_Fiction",
             "https://www.goodreads.com/list/show/1938.What_To_Read_Next"]

list_num_pages = [232, 291, 70, 66, 35, 132]

Info_Book_Url = pd.DataFrame({"Book_Titles": [], "Book_Urls": [], "Author_Names": [], "Author_Urls": []})

for i in range(len(list_album)):
    for page in tqdm(range(1, list_num_pages[i] + 1)):
        Info_Book_Url = pd.concat([Info_Book_Url, get_Url_Book_by_Album(list_album[i], page)], 
                                      ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████████| 133/133 [11:15<00:00,  5.08s/it]


In [ ]:
# Info_Book_Url = pd.DataFrame({"Book_Titles": [], "Book_Urls": [], "Author_Names": [], "Author_Urls": []})

url_album = "https://www.goodreads.com/list/show/10198"
num_pages = 34

for page in tqdm(range(1, num_pages + 1)):
    Info_Book_Url = pd.concat([Info_Book_Url, get_Url_Book_by_Album(url_album, page)], 
                                    ignore_index = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [02:50<00:00,  5.00s/it]


In [ ]:
Info_Book_Url.drop_duplicates(subset = ["Book_Titles", "Book_Urls", "Author_Names"], inplace = True)
# Xoá đi những quyển sách bị trùng tên và trùng tác giả
Info_Book_Url

,Book_Titles,Book_Urls,Author_Names,Author_Urls
0,Besos entre líneas,https://www.goodreads.com/book/show/28799408-b...,May R. Ayamonte,https://www.goodreads.com/author/show/7182094....
1,Birth Control Is Sinful in the Christian Marri...,https://www.goodreads.com/book/show/6324600-bi...,Eliyzabeth Yanne Strong-Anderson,https://www.goodreads.com/author/show/3984059....
2,A Parent's Guide to Preventing Homosexuality,https://www.goodreads.com/book/show/958263.A_P...,Joseph Nicolosi,https://www.goodreads.com/author/show/485543.J...
3,Coming Out Straight: Understanding and Healing...,https://www.goodreads.com/book/show/2574850-co...,Richard Cohen,https://www.goodreads.com/author/show/15306662...
4,Dianetics: The Modern Science of Mental Health,https://www.goodreads.com/book/show/179166.Dia...,L. Ron Hubbard,https://www.goodreads.com/author/show/33503.L_...
...,...,...,...,...
3762,B,https://www.goodreads.com/book/show/13123245-b,Sarah Kay,https://www.goodreads.com/author/show/11377.Sa...
3763,Hellados,https://www.goodreads.com/book/show/9487794-he...,Nodar Dumbadze,https://www.goodreads.com/author/show/1834895....
3764,Наполеонов обоз. Книга 2. Белые лошади,https://www.goodreads.com/book/show/44796327-2,Dina Rubina,https://www.goodreads.com/author/show/235809.D...
3765,Adventure Time #4,https://www.goodreads.com/book/show/14761409-a...,Ryan North,https://www.goodreads.com/author/show/45763.Ry...


## Lựa chọn theo tác giả

In [ ]:
list_author = Info_Book_Url.Author_Urls.unique()
list_author

array(['https://www.goodreads.com/author/show/7182094.May_R_Ayamonte',
       'https://www.goodreads.com/author/show/3984059.Eliyzabeth_Yanne_Strong_Anderson',
       'https://www.goodreads.com/author/show/485543.Joseph_Nicolosi',
       ..., 'https://www.goodreads.com/author/show/11377.Sarah_Kay',
       'https://www.goodreads.com/author/show/235809.Dina_Rubina',
       'https://www.goodreads.com/author/show/17990067.Joshua_Barkman'],
      dtype=object)

In [ ]:
def get_Url_Book_by_Author(url): # Lấy Url Book theo tác giả
    url = url.replace("show", "list") + "?page=1&per_page=300"
    source = requests.get(url)
    soup =   BeautifulSoup(source.text, 'html.parser')

    try:
        temp = soup.find("table", {"class": "tableList"})
        temp = temp.findAll("tr", {"itemtype": "http://schema.org/Book"})
    except:
        return None
    
    Book_Titles, Book_Urls, Author_Names, Author_Urls = create_data_url()
    
    for t in temp:
        Book_Titles.append(str(t.find("a", {"class": "bookTitle"}).find("span").contents[0]))
        Book_Urls.append(str("https://www.goodreads.com" + t.find("a", {"class": "bookTitle"}).get("href")))
        Author_Names.append(str(t.find("a", {"class": "authorName"}).find("span").contents[0]))
        Author_Urls.append(str(t.find("a", {"class": "authorName"}).get("href")))

        
    return pd.DataFrame({"Book_Titles": Book_Titles, "Book_Urls": Book_Urls, "Author_Names": Author_Names, "Author_Urls": Author_Urls})

In [ ]:
Info_Book_Url_temp = pd.DataFrame({"Book_Titles": [], "Book_Urls": [], "Author_Names": [], "Author_Urls": []})
for url in tqdm(list_author[4071:5000]):
    Info_Book_Url_temp = pd.concat([Info_Book_Url_temp, get_Url_Book_by_Author(url)], ignore_index = True)
    gc.collect()
    
Info_Book_Url = pd.concat([Info_Book_Url, Info_Book_Url_temp], ignore_index = True)

100%|██████████████████████████████████████████████████████████████████████████████| 929/929 [1:00:16<00:00,  3.89s/it]


In [ ]:
Info_Book_Url.drop_duplicates(subset = ["Book_Titles", "Book_Urls", "Author_Names"], inplace = True)
# Xoá đi những quyển sách bị trùng tên và trùng tác giả
Info_Book_Url

,Book_Titles,Book_Urls,Author_Names,Author_Urls
0,Ways of Seeing (Paperback),https://www.goodreads.com/book/show/2784.Ways_...,John Berger,https://www.goodreads.com/author/show/29919.Jo...
1,The Story of Art (Hardcover),https://www.goodreads.com/book/show/222078.The...,E.H. Gombrich,https://www.goodreads.com/author/show/167112.E...
2,The New Drawing on the Right Side of the Brain...,https://www.goodreads.com/book/show/627206.The...,Betty Edwards,https://www.goodreads.com/author/show/42010.Be...
3,Steal Like an Artist: 10 Things Nobody Told Yo...,https://www.goodreads.com/book/show/13099738-s...,Austin Kleon,https://www.goodreads.com/author/show/2985039....
4,Wall and Piece (Paperback),https://www.goodreads.com/book/show/114683.Wal...,Banksy,https://www.goodreads.com/author/show/28811.Ba...
...,...,...,...,...
231152,Œuvres (annoté) (Littérature),https://www.goodreads.com/book/show/56361640-u...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
231153,Lettres retrouvées,https://www.goodreads.com/book/show/55467944-l...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
231154,I fidanzati di tredici anni e altre poesie,https://www.goodreads.com/book/show/44776821-i...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
231155,Tutte le poesie,https://www.goodreads.com/book/show/44776561-t...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...


In [ ]:
Info_Book_Url.to_csv("Info_Book_Url.csv", index = False, encoding = "utf-8")
# Lưu những đường link dẫn đến book vào file Info_Book_Url.csv

In [ ]:
Info_Book_Url = pd.read_csv("../data/Info_Book_Url.csv") # Tiếp tục thực hiện từ đây
Info_Book_Url

,Book_Titles,Book_Urls,Author_Names,Author_Urls
0,Ways of Seeing (Paperback),https://www.goodreads.com/book/show/2784.Ways_...,John Berger,https://www.goodreads.com/author/show/29919.Jo...
1,The Story of Art (Hardcover),https://www.goodreads.com/book/show/222078.The...,E.H. Gombrich,https://www.goodreads.com/author/show/167112.E...
2,The New Drawing on the Right Side of the Brain...,https://www.goodreads.com/book/show/627206.The...,Betty Edwards,https://www.goodreads.com/author/show/42010.Be...
3,Steal Like an Artist: 10 Things Nobody Told Yo...,https://www.goodreads.com/book/show/13099738-s...,Austin Kleon,https://www.goodreads.com/author/show/2985039....
4,Wall and Piece (Paperback),https://www.goodreads.com/book/show/114683.Wal...,Banksy,https://www.goodreads.com/author/show/28811.Ba...
...,...,...,...,...
225558,Œuvres (annoté) (Littérature),https://www.goodreads.com/book/show/56361640-u...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
225559,Lettres retrouvées,https://www.goodreads.com/book/show/55467944-l...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
225560,I fidanzati di tredici anni e altre poesie,https://www.goodreads.com/book/show/44776821-i...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...
225561,Tutte le poesie,https://www.goodreads.com/book/show/44776561-t...,Raymond Radiguet,https://www.goodreads.com/author/show/423070.R...


In [ ]:
list_url_books = Info_Book_Url.Book_Urls.to_list() # Lấy danh sách url book ra để crawl từng url
list_url_books

['https://www.goodreads.com/book/show/2784.Ways_of_Seeing',
 'https://www.goodreads.com/book/show/222078.The_Story_of_Art',
 'https://www.goodreads.com/book/show/627206.The_New_Drawing_on_the_Right_Side_of_the_Brain',
 'https://www.goodreads.com/book/show/13099738-steal-like-an-artist',
 'https://www.goodreads.com/book/show/114683.Wall_and_Piece',
 'https://www.goodreads.com/book/show/615570.The_Artist_s_Way',
 'https://www.goodreads.com/book/show/2865.Girl_with_a_Pearl_Earring',
 'https://www.goodreads.com/book/show/187633.Art_and_Fear',
 'https://www.goodreads.com/book/show/567616.The_Art_Book',
 'https://www.goodreads.com/book/show/857502.Concerning_the_Spiritual_in_Art',
 'https://www.goodreads.com/book/show/6988014-seven-days-in-the-art-world',
 'https://www.goodreads.com/book/show/395233.The_Letters_of_Vincent_van_Gogh',
 'https://www.goodreads.com/book/show/17333223-the-goldfinch',
 'https://www.goodreads.com/book/show/10505.History_of_Beauty',
 'https://www.goodreads.com/book/s

In [ ]:
def clean_html(raw_html): # Khi crawl Description, ta sẽ xoá đi những tag html: <a>, <p>,....
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def convert_number(num): # Chuyển hoá string -> int: 9,500 -> 9500, 1,234,244 -> 1234244
    num = num.replace(",","")
    return float(num)

In [ ]:
def get_Detail_Book(url): # Lấy chi tiết từng url book

    source = requests.get(url)
    soup =   BeautifulSoup(source.text, 'html.parser')

    temp = soup.find("div", {"class": "last col stacked"})
    if temp == None: # Trường hợp trang bị mất hoặc sách đã bị xoá
        return None
    
    try:
        Image_url = str(temp.find("div", {"class": "bookCoverPrimary"}).find("img").get("src"))
    except:
        Image_url = None

    temp = temp.find("div", {"class": "last col"})
    if temp == None: # Trường hợp không hiển thị thông tin tiêu đề sách
        return None
    
    try:
        Book_Title = str(temp.find("h1", {"id":"bookTitle"}).contents[0].strip())
    except:
        Book_Title = None

    try:
        Author = temp.find("a", {"class": "authorName"})
        Author_Url = str(Author.get("href"))
        Author_Name = str(Author.find("span").contents[0])
    except:
        Author_Url, Author_Name = None, None

    try:
        Description = temp.find("div", {"id": "description"}).findAll("span")
        if len(Description) > 1:
            Description = clean_html(str(Description[1]))
        else:
            Description = clean_html(str(Description[0]))
    except:
        Description = None

    try:
        Rating = convert_number(str(temp.find("span", {"itemprop": "ratingValue"}).contents[0].strip()))
    except:
        Rating = None

    try:
        Rating_Counts = convert_number(str(temp.find("meta", {"itemprop": "ratingCount"}).get("content")))
    except:
        Rating_Counts = None

    try:
        Reviews_Counts = convert_number(str(temp.find("meta", {"itemprop": "reviewCount"}).get("content")))
    except:
        Reviews_Counts = None

    temp = soup.find("div", {"id": "details"})
    if temp == None: # Trường hợp sách không có thông tin chi tiết
        return None
    
    try:
        Num_Pages = convert_number(str(temp.find("span", {"itemprop": "numberOfPages"}).contents[0])[:-5])
    except:
        Num_Pages = None

    try:
        Book_Format = str(temp.find("span", {"itemprop": "bookFormat"}).contents[0])
    except:
        Book_Format = None

    try:
        Publish = temp.findAll("div", {"class":"row"})
        Publish = [k for k in Publish if "Published" in str(k)][0].contents[0].strip().split("Published")[1].split("by")
        if len(Publish) > 1: 
            Year_publish = Publish[0].strip()
            Publisher = Publish[1].strip()
        else: # Trường hợp có publisher mà không có year_publish và ngược lại
            Year_publish = Publish[0].strip()
            Publisher = None
    except:
        Year_publish = None
        Publisher = None

    InfoDetail = temp.find("div", {"id":"bookDataBox"}).findAll("div", {"class": "clearFloats"})
    try:
        ISBN = [k for k in InfoDetail if "ISBN" in str(k)][0].find("div", {"class": "infoBoxRowItem"}).contents[0].strip()
        ISBN = str(ISBN)
    except:
        ISBN = None

    try:
        Language = [k for k in InfoDetail if "Edition Language" in str(k)][0].find("div", {"class": "infoBoxRowItem"}).contents[0].strip()
        Language = str(Language)
    except:
        Language = None

    temp = soup.find("div", {"class": "rightContainer"})
    Genres = {}
    if temp != None:
        temp = temp.findAll("div", {"class": "stacked"})[1].findAll("div", {"class": "elementList"})
        
        for element in temp: # Sắp xếp Genres theo kiểu dictionaries: {"classic": 10, "romantic": 5}
            genre = element.find("div", {"class": "left"}).find("a").contents[0]
            users_number = convert_number(element.find("div", {"class": "right"}).find("a").contents[0][:-6])
            Genres[genre] = users_number
    else:
        Genres = None

    Genres = json.dumps(Genres)
    
    return pd.DataFrame({"Book_Title": Book_Title, "Author_Name": Author_Name, "Author_Url": Author_Url,
                        "Description": Description, "Rating": Rating, "Rating_Counts": Rating_Counts,
                        "Reviews_Counts": Reviews_Counts, "Num_Pages": Num_Pages, "Book_Format" :Book_Format,
                        "Time_publish": Year_publish, "Publisher": Publisher, "ISBN": ISBN, 
                        "Language": Language, "Genres": Genres, "Book_Url": url}, index=[0])


In [ ]:
def create_null_dataframe_info_book_detail(): # Khởi tạo data ban đầu
    return pd.DataFrame({"Book_Title": [], "Author_Name": [], "Author_Url": [],
                        "Description": [], "Rating": [], "Rating_Counts": [],
                        "Reviews_Counts": [], "Num_Pages": [], "Book_Format" :[],
                        "Time_publish": [], "Publisher": [], "ISBN": [], 
                        "Language": [], "Genres": [], "Book_Url": []})

In [ ]:
info_book_detail = create_null_dataframe_info_book_detail()
for book_url in tqdm(list_url_books[:3]):
    info_book_detail = pd.concat([info_book_detail, get_Detail_Book(book_url)], ignore_index = True)
    
info_book_detail

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.21s/it]


,Book_Title,Author_Name,Author_Url,Description,Rating,Rating_Counts,Reviews_Counts,Num_Pages,Book_Format,Time_publish,Publisher,ISBN,Language,Genres,Book_Url
0,Ways of Seeing,John Berger,https://www.goodreads.com/author/show/29919.Jo...,John Berger’s Classic Text on ArtJohn Berger's...,3.86,259355.0,1616.0,176.0,Paperback,1990,Penguin,0140135154,English,"{""Art"": 145.0, ""Nonfiction"": 1127.0, ""Philosop...",https://www.goodreads.com/book/show/2784.Ways_...
1,The Story of Art,E.H. Gombrich,https://www.goodreads.com/author/show/167112.E...,This text is the 16th revised and updated edit...,3.93,317640.0,945.0,688.0,Hardcover,April 9th 1995,Phaidon Press,071483355X,English,"{""Art"": 26.0, ""Nonfiction"": 559.0, ""History"": ...",https://www.goodreads.com/book/show/222078.The...
2,The New Drawing on the Right Side of the Brain,Betty Edwards,https://www.goodreads.com/author/show/42010.Be...,When Drawing on the Right Side of the Brain wa...,3.86,300600.0,833.0,291.0,Paperback,August 30th 1999,Tarcher,0874774241,English,"{""Art"": 26.0, ""Nonfiction"": 540.0, ""Reference""...",https://www.goodreads.com/book/show/627206.The...


## Tiếp theo, ta sẽ sử dụng multiprocessing để crawl dữ liệu nhanh hơn
+ Ý tưởng: Ta sẽ áp dụng multiprocessing để đào dữ liệu trong 1 danh sách nhất định, nguyên nhân do goodreads sẽ làm chậm tốc độ khi get request trang web nên ta sẽ crawl theo từng đợt 100 url. Cụ thể, ta sẽ cho vòng lặp khoảng crawl 10k đường dẫn, mỗi vòng chọn 100 url để crawl.
+ Thời gian crawl: 10000 url sách / 60 phút
+ Tổng thời gian crawl: 10 tiếng

In [ ]:
min_index = 90000
max_index = 100000
# Chạy file crawl (có chứa multiprocessing)
for index in tqdm(range(min_index, max_index, 100)):
    %run crawl_info_book_detail.py -min {str(index)} -max {str(index + 100)}
    time.sleep(2)
    gc.collect()

## Sau khi crawl xong ta sẽ đánh giá lại dữ liệu, xoá đi những hàng bị trùng lắp

In [ ]:
a = pd.read_csv("../data/info_book_detail.csv", encoding = "utf-8")
a.drop_duplicates(subset = ["Book_Title", "Author_Name", "Rating", "Num_Pages"], inplace = True)
# Xoá đi những rows bị trùng
a.reset_index(inplace = True, drop = True)
a

,Book_Title,Author_Name,Author_Url,Description,Rating,Rating_Counts,Reviews_Counts,Num_Pages,Book_Format,Time_publish,Publisher,ISBN,Language,Genres,Book_Url
0,Ways of Seeing,John Berger,https://www.goodreads.com/author/show/29919.Jo...,John Berger’s Classic Text on ArtJohn Berger's...,3.86,258532.0,1608.0,176.0,Paperback,1990,Penguin,0140135154,English,"{""Art"": 145.0, ""Nonfiction"": 1124.0, ""Philosop...",https://www.goodreads.com/book/show/2784.Ways_...
1,The Story of Art,E.H. Gombrich,https://www.goodreads.com/author/show/167112.E...,This text is the 16th revised and updated edit...,3.93,316875.0,940.0,688.0,Hardcover,April 9th 1995,Phaidon Press,071483355X,English,"{""Art"": 25.0, ""Nonfiction"": 556.0, ""History"": ...",https://www.goodreads.com/book/show/222078.The...
2,The New Drawing on the Right Side of the Brain,Betty Edwards,https://www.goodreads.com/author/show/42010.Be...,When Drawing on the Right Side of the Brain wa...,3.86,300196.0,830.0,291.0,Paperback,August 30th 1999,Tarcher,0874774241,English,"{""Art"": 26.0, ""Nonfiction"": 541.0, ""Reference""...",https://www.goodreads.com/book/show/627206.The...
3,Steal Like an Artist: 10 Things Nobody Told Yo...,Austin Kleon,https://www.goodreads.com/author/show/2985039....,"You don’t need to be a genius, you just need t...",3.93,203239.0,5641.0,160.0,Paperback,February 28th 2012,Workman Publishing Company,0761169253,English,"{""Nonfiction"": 1714.0, ""Art"": 66.0, ""Language""...",https://www.goodreads.com/book/show/13099738-s...
4,Wall and Piece,Banksy,https://www.goodreads.com/author/show/28811.Ba...,"Banksy, Britain's now-legendary ""guerilla"" str...",3.85,205180.0,874.0,240.0,Paperback,2006,Century,1844137872,English,"{""Art"": 17.0, ""Nonfiction"": 357.0, ""Politics"":...",https://www.goodreads.com/book/show/114683.Wal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94492,Kaleidoszkóp,Danielle Steel,https://www.goodreads.com/author/show/14255.Da...,"1943-ban Franciaországban, a szövetségesek par...",5.00,1.0,0.0,372.0,Paperback,2016,Maecenas,9789632030937,Hungarian,NaN,https://www.goodreads.com/book/show/50284635-k...
94493,Kochanka,Danielle Steel,https://www.goodreads.com/author/show/14255.Da...,NaN,5.00,1.0,0.0,NaN,paperback,January 1st 2017,NaN,8324038310,NaN,NaN,https://www.goodreads.com/book/show/53231683-k...
94494,"Omnibus - De kroonprinses, Geheimen",Danielle Steel,https://www.goodreads.com/author/show/14255.Da...,De kroonprinsesChristianna lijkt een moderne j...,5.00,1.0,0.0,600.0,Kindle Edition,July 11th 2019,Luitingh Sijthoff,NaN,NaN,NaN,https://www.goodreads.com/book/show/45855739-o...
94495,Danielle steel books Set Series 1 : 8 books co...,Danielle Steel,https://www.goodreads.com/author/show/14255.Da...,Danielle steel collection 8 books set includes...,5.00,1.0,0.0,NaN,Paperback,NaN,Sphere,9123663618,NaN,NaN,https://www.goodreads.com/book/show/40814010-d...


In [ ]:
a.to_csv("../data/info_book_detail.csv", index = False, encoding = "utf-8")

# Tìm User Info Rating (Recommendation System)

## Mục tiêu:
#### Lấy dữ liệu rating của người dùng đối với quyển sách, phương hướng tiếp cận gồm 2 hướng khác nhau:
+ Lấy dữ liệu từ trang của người dùng (dựa vào user_url) để lấy những quyển sách người dùng đã đánh giá
+ Lấy dữ liệu từ trang web của trang sách (dựa vào book_url) để lấy đánh giá từ bình luận (comment) của người dùng đối với sách
+ Lý do: Nguyên nhân của việc đào dữ liệu từ 2 phía này là để tăng tỉ lệ số lượng books/user và tỉ lệ số lượng users/book cao. Dữ liệu này được sử dụng trong mô hình Matrix Factorization nên cần tỉ lệ tốt để dự đoán rating của người dùng

## Phương hướng 1: Lấy dữ liệu từ trang người dùng

In [ ]:
def create_user_data(): # Khởi tạo data ban đầu
    book_title = [] # Tiêu đề sách
    book_url = [] # Url sách
    username = [] # Tên người dùng
    user_url = [] # Url user
    user_rating = [] # Người dùng đánh giá (1-5)
    return book_title, book_url, username, user_url, user_rating

In [ ]:
def get_user_from_comments(book): # Lấy thông tin rating của user thông qua comment của mỗi sách
    
    source = requests.get(book)
    soup =   BeautifulSoup(source.text, 'html.parser')

    try: 
        temp = soup.find("div", {"id": "bookReviews"})
    except:
        return None

    book_title, book_url, username, user_url, user_rating = create_user_data()

    reviews = temp.find_all('div', class_ = 'friendReviews elementListBrown')

    for review in reviews:
        try:
            book_title.append(str(soup.find("h1", {"id":"bookTitle"}).contents[0].strip()))
        except:
            book_title.append(None)
        
        # user_url / username
        info_user = review.find('a')
        user_url.append("https://www.goodreads.com" + str(info_user['href']))
        username.append(str(info_user['title']))
        
        # user_rating
        user_rating.append(len(review.find_all('span', class_="staticStar p10")))

        # book_url
        book_url.append(book)
        
    return pd.DataFrame({"url_book": book_url, "book_title": book_title, "username": username, 
                        "user_url": user_url, "user_rating": user_rating})

In [ ]:
user_rating_data = pd.DataFrame({"url_book": [], "book_title": [], "username": [], 
                                "user_url": [], "user_rating": []})

for book_url in tqdm(a.Book_Url[:100]): # Test với 100 url_book ban đầu
    user_rating_data = pd.concat([user_rating_data, get_user_from_comments(book_url)], ignore_index = True)
    
user_rating_data

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:27<00:00,  2.68s/it]


,url_book,book_title,username,user_url,user_rating
0,https://www.goodreads.com/book/show/2784.Ways_...,Ways of Seeing,Trevor,https://www.goodreads.com/user/show/175635-trevor,5.0
1,https://www.goodreads.com/book/show/2784.Ways_...,Ways of Seeing,Ahmad Sharabiani,https://www.goodreads.com/user/show/614778-ahm...,4.0
2,https://www.goodreads.com/book/show/2784.Ways_...,Ways of Seeing,Justin Evans,https://www.goodreads.com/user/show/2184529-ju...,3.0
3,https://www.goodreads.com/book/show/2784.Ways_...,Ways of Seeing,Pierce,https://www.goodreads.com/user/show/239699-pierce,4.0
4,https://www.goodreads.com/book/show/2784.Ways_...,Ways of Seeing,Steven Godin,https://www.goodreads.com/user/show/26706841-s...,4.0
...,...,...,...,...,...
2995,https://www.goodreads.com/book/show/4865.How_t...,How to Win Friends and Influence People,booklady,https://www.goodreads.com/user/show/350218-boo...,5.0
2996,https://www.goodreads.com/book/show/4865.How_t...,How to Win Friends and Influence People,11811 (Eleven),https://www.goodreads.com/user/show/4460829-11...,2.0
2997,https://www.goodreads.com/book/show/4865.How_t...,How to Win Friends and Influence People,Keertana,https://www.goodreads.com/user/show/7584700-ke...,3.0
2998,https://www.goodreads.com/book/show/4865.How_t...,How to Win Friends and Influence People,Kendel Christensen,https://www.goodreads.com/user/show/5081086-ke...,5.0


In [ ]:
user_rating_data.to_csv("../data/user_rating_beginner.csv", index = False, encoding = "utf-8") # Lưu trữ dataframe ban đầu

In [ ]:
user_rating_data = pd.read_csv("../data/user_rating_beginner.csv", encoding = "utf-8")

In [ ]:
# Lấy nickname của người dùng
def get_last_element(a):
    return a[-1]

# Mục tiêu: Dựa vào nickname của người dùng để áp dụng cho phương hướng 2 là lấy dữ liệu đánh giá của sách dựa vào username
list_nickname = user_rating_data.user_url.str.split("/").apply(get_last_element) # Lấy nickname của user
list_nickname

0                    175635-trevor
1          614778-ahmad-sharabiani
2             2184529-justin-evans
3                    239699-pierce
4            26706841-steven-godin
                   ...            
2995               350218-booklady
2996          4460829-11811-eleven
2997              7584700-keertana
2998    5081086-kendel-christensen
2999       80152812-maciej-nowicki
Name: user_url, Length: 3000, dtype: object

## Phương hướng 2: Lấy danh sách đánh giá của người dùng dựa trên trang web cá nhân của user dựa vào username

In [ ]:
def create_rating_data():
    url_book = []
    book_title = []
    username = []
    user_url = []
    user_rating = []
    return url_book, book_title, username, user_url, user_rating

In [ ]:
def get_user_rating_book(nickname):
    
    source = requests.get("https://www.goodreads.com/review/list/{}?page=1&print=true&shelf=read&sort=date_added&view=table".format(nickname))
    soup =   BeautifulSoup(source.text, 'html.parser')
    
    try: # Trường hợp bị lock, ví dụ như librarian không public thông tin cá nhân
        temp = soup.find("div", {"id": "header"}).findAll("a")
    except:
        return None
    temp_username = str(temp[1].contents[0])
    temp_user_url = str(temp[1].get("href"))

    try: # Trang của web goodreads: Back 1 2 3 ... 9 Next
        pages = int(soup.find("div", {"id": "reviewPagination"}).findAll("a")[-2].contents[0])
    except: # Trường hợp số trang chỉ có 1 hoặc không có mục trang
        pages = 1
        
    url_book, book_title, username, user_url, user_rating = create_rating_data()

    for page in tqdm(range(1, min(100, pages + 1))): # Lấy tối đa 100 trang
        source = requests.get("https://www.goodreads.com/review/list/{}?page={}&print=true&shelf=read&sort=date_added&view=table".format(nickname, page))
        soup =   BeautifulSoup(source.text, 'html.parser')

        try: # Lấy danh sách các quyển sách
            temp = soup.find("tbody", {"id": "booksBody"})
            temp = temp.findAll("tr", {"class": "bookalike review"})
        except:
            continue
            
        for (id, book) in enumerate(temp):
            try:
                url_book.append("https://www.goodreads.com" + str(book.find("td", {"class": "field title"}).find("a").get("href")))
            except:
                url_book.append(None)
            try:
                book_title.append(str(book.find("td", {"class": "field title"}).find("a").contents[0]).strip())
            except:
                book_title.append(None)
            try:
                user_rating.append(len(book.find("td", {"class": "field rating"}).findAll("span", {"class": "staticStar p10"})))
            except:
                user_rating.append(None)
                
            username.append(temp_username)
            user_url.append("https://www.goodreads.com" + temp_user_url)

    return pd.DataFrame({"url_book": url_book, "book_title": book_title, "username": username, 
                        "user_url": user_url, "user_rating": user_rating})

In [ ]:
user_rating_book = get_user_rating_book("614778-ahmad-sharabiani")
user_rating_book

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [04:48<00:00,  2.92s/it]


,url_book,book_title,username,user_url,user_rating
0,https://www.goodreads.com/book/show/9834882-th...,The Crscent Moon and Stray Birds / ماه نو و مر...,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4
1,https://www.goodreads.com/book/show/55359022-a...,A Promised Land,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,0
2,https://www.goodreads.com/book/show/13600697,اولریکا و هشت داستان دیگر,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4
3,https://www.goodreads.com/book/show/53025558,صد سال تنهایی,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,3
4,https://www.goodreads.com/book/show/171970.Alamut,Alamut,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,0
...,...,...,...,...,...
1975,https://www.goodreads.com/book/show/30636761,چند معراجنامه,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,5
1976,https://www.goodreads.com/book/show/30547912,اگر خواستگار نیاید,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,3
1977,https://www.goodreads.com/book/show/26097874,(ابراهیم (قصص الانبیاء,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,5
1978,https://www.goodreads.com/book/show/1419086._,چند غزل از حافظ,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,5


In [ ]:
user_rating_book.to_csv("user_rating_book.csv", index = False, encoding = "utf-8")

In [ ]:
min_index = 800
max_index = 900

for nickname in tqdm(list_nickname[min_index : max_index]):
    %run crawl_user_rating.py -name {str(nickname)}
    gc.collect()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [39:35<00:00, 23.76s/it]


In [ ]:
user_rating_book = pd.read_csv("../data/user_rating_book.csv", encoding = "utf-8", index_col = False)
# Xoá data bị trùng lắp
user_rating_book.drop_duplicates(subset = ["url_book", "username", "user_url"], inplace = True)
# Xoá data có rating = 0, nguyên nhân do người dùng chưa hoàn thành việc đọc sách
user_rating_book.drop(user_rating_book[user_rating_book.user_rating == 0].index, inplace = True)
user_rating_book.reset_index(inplace = True, drop = True)
user_rating_book

,url_book,book_title,username,user_url,user_rating
0,https://www.goodreads.com/book/show/9834882-th...,The Crscent Moon and Stray Birds / ماه نو و مر...,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0
1,https://www.goodreads.com/book/show/13600697,اولریکا و هشت داستان دیگر,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0
2,https://www.goodreads.com/book/show/53025558,صد سال تنهایی,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,3.0
3,https://www.goodreads.com/book/show/13628812-t...,The Rent Collector,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0
4,https://www.goodreads.com/book/show/21487668-j...,James Baldwin: The Last Interview and Other Co...,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0
...,...,...,...,...,...
321153,https://www.goodreads.com/book/show/5326.A_Chr...,A Christmas Carol,Books,https://www.goodreads.com/review/list/88896249...,4.0
321154,https://www.goodreads.com/book/show/17245.Dracula,Dracula,Books,https://www.goodreads.com/review/list/88896249...,5.0
321155,https://www.goodreads.com/book/show/51496.The_...,The Strange Case of Dr. Jekyll and Mr. Hyde,Books,https://www.goodreads.com/review/list/88896249...,4.0
321156,https://www.goodreads.com/book/show/170448.Ani...,Animal Farm,Books,https://www.goodreads.com/review/list/88896249...,4.0


In [ ]:
user_rating_book.to_csv("../data/user_rating_book.csv", index = False, encoding = "utf-8")

### Xử lý dữ liệu đánh giá của người dùng
+ Do crawl theo user_url và book_url nên 2 column này độc lập và được xem như ID, do đó, ta dựa vào 2 column này để lấy ID

In [ ]:
user_rating_book["book_id"] = pd.factorize(user_rating_book.url_book)[0]
user_rating_book["user_id"] = pd.factorize(user_rating_book.user_url)[0]

In [ ]:
user_rating_book

,url_book,book_title,username,user_url,user_rating,book_id,user_id
0,https://www.goodreads.com/book/show/9834882-th...,The Crscent Moon and Stray Birds / ماه نو و مر...,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,0,0
1,https://www.goodreads.com/book/show/13600697,اولریکا و هشت داستان دیگر,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,1,0
2,https://www.goodreads.com/book/show/53025558,صد سال تنهایی,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,3.0,2,0
3,https://www.goodreads.com/book/show/13628812-t...,The Rent Collector,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,3,0
4,https://www.goodreads.com/book/show/21487668-j...,James Baldwin: The Last Interview and Other Co...,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,4,0
...,...,...,...,...,...,...,...
321153,https://www.goodreads.com/book/show/5326.A_Chr...,A Christmas Carol,Books,https://www.goodreads.com/review/list/88896249...,4.0,5757,2195
321154,https://www.goodreads.com/book/show/17245.Dracula,Dracula,Books,https://www.goodreads.com/review/list/88896249...,5.0,2647,2195
321155,https://www.goodreads.com/book/show/51496.The_...,The Strange Case of Dr. Jekyll and Mr. Hyde,Books,https://www.goodreads.com/review/list/88896249...,4.0,3427,2195
321156,https://www.goodreads.com/book/show/170448.Ani...,Animal Farm,Books,https://www.goodreads.com/review/list/88896249...,4.0,9055,2195


In [ ]:
temp = user_rating_book.groupby(["book_id"]).count() # Đếm số lượng book_id/user
list_book_id_for_ML = temp[temp.url_book > 30].index # Chọn những book_id có tỉ lệ > 30

In [ ]:
temp = user_rating_book.groupby(["user_id"]).count() # Đếm số lượng user_id/book
list_user_id_for_ML = temp[temp.url_book > 30].index # Chọn những user_id có tỉ lệ > 30

In [ ]:
user_rating_book = user_rating_book[user_rating_book.book_id.isin(list_book_id_for_ML)]
user_rating_book = user_rating_book[user_rating_book.user_id.isin(list_user_id_for_ML)]
user_rating_book

,url_book,book_title,username,user_url,user_rating,book_id,user_id
22,https://www.goodreads.com/book/show/8664353-un...,"Unbroken: A World War II Story of Survival, Re...",Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,22,0
23,https://www.goodreads.com/book/show/9418327-bo...,Bossypants,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,23,0
24,https://www.goodreads.com/book/show/1617.Night,Night,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,24,0
68,https://www.goodreads.com/book/show/32075671-t...,The Hate U Give,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,68,0
92,https://www.goodreads.com/book/show/38501.Ham_...,Ham on Rye,Ahmad,https://www.goodreads.com/user/show/614778-ahm...,4.0,92,0
...,...,...,...,...,...,...,...
321141,https://www.goodreads.com/book/show/4667024-th...,The Help,Books,https://www.goodreads.com/review/list/88896249...,5.0,7514,2195
321142,https://www.goodreads.com/book/show/23692271-s...,Sapiens: A Brief History of Humankind,Books,https://www.goodreads.com/review/list/88896249...,5.0,4341,2195
321153,https://www.goodreads.com/book/show/5326.A_Chr...,A Christmas Carol,Books,https://www.goodreads.com/review/list/88896249...,4.0,5757,2195
321154,https://www.goodreads.com/book/show/17245.Dracula,Dracula,Books,https://www.goodreads.com/review/list/88896249...,5.0,2647,2195


In [ ]:
user_rating_book.to_csv("../data/user_rating_book_filter_per_30.csv", index = False, encoding = "utf-8")

In [ ]:
info_book_detail_per_30 = create_null_dataframe_info_book_detail()
for book_url in tqdm(user_rating_book.url_book):
    info_book_detail_per_30 = pd.concat([info_book_detail_per_30, get_Detail_Book(book_url)], ignore_index = True)

In [ ]:
info_book_detail_per_30.to_csv("../data/info_book_detail_per_30.csv", index = False, encoding = "utf-8")